In [1]:
import gymnasium as gym
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import random
import math
from tqdm.notebook import trange

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
class CartPole:
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        self.action_size = self.env.action_space.n

    def __repr__(self):
        return 'CartPole-v1'

    def get_initial_state(self):
        observation, info = self.env.reset()
        valid_locations = self.get_valid_locations(observation)
        reward = 0
        is_terminal = False
        return observation, valid_locations, reward, is_terminal

    def step(self, action):
        observation, reward, is_terminal, _, _ = self.env.step(action)
        valid_locations = self.get_valid_locations(observation)
        return observation, valid_locations, reward, is_terminal

    def get_valid_locations(self, observation):
        return self.action_size

    def get_canonical_state(self, hidden_state, player):
        return hidden_state

    def get_encoded_observation(self, observation):
        return observation    
    
    def get_opponent_player(self, player):
        return player

    def get_opponent_value(self, value):
        return value


In [3]:
class ReplayBuffer:
    def __init__(self, args, game):
        self.memory = []
        self.trajectories = []
        self.args = args
        self.game = game

    def __len__(self):
        return len(self.memory)

    def empty(self):
        self.memory = []

    def build_trajectories(self):
        self.trajectories = []
        for i in range(len(self.memory)):
            observation, action, policy, reward, game_idx, is_terminal = self.memory[i]
            if is_terminal:
                action = np.random.choice(self.game.action_size)

            current_value = []
            for n in range(1, self.args['N'] + 1):
                if i + n < len(self.memory) and self.memory[i + n][4] == game_idx:
                    _, _, _, reward, _, _ = self.memory[i + n]
                    current_value.append(reward)
                else:
                    current_value.append(0)
            value = np.sum([self.args['gamma'] ** n * current_value[n - 1] for n in range(1, self.args['N'] + 1)])

            policy_list, action_list, value_list, reward_list = [policy], [action], [value], [reward]

            for k in range(1, self.args['K'] + 1):
                if i + k < len(self.memory) and self.memory[i + k][4] == game_idx:
                    _, action, policy, reward, _, is_terminal = self.memory[i + k]
                    if is_terminal:
                        action = np.random.choice(self.game.action_size)
                    action_list.append(action)
                    policy_list.append(policy)
                    reward_list.append(reward)

                    current_value = []
                    for n in range(1, self.args['N'] + 1):
                        if i + k + n < len(self.memory) and self.memory[i + k + n][4] == game_idx:
                            _, _, _, reward, _, _ = self.memory[i + k + n]
                            current_value.append(reward)
                        else:
                            current_value.append(0)
                    value = np.sum([self.args['gamma'] ** n * current_value[n - 1] for n in range(1, self.args['N'] + 1)])

                    value_list.append(value)

                else:
                    action_list.append(np.random.choice(self.game.action_size))
                    policy_list.append(policy_list[-1])
                    value_list.append(self.game.get_opponent_value(1) * value_list[-1])
                    reward_list.append(self.game.get_opponent_value(1) * reward_list[-1])

            policy_list = np.stack(policy_list)
            self.trajectories.append((observation, action_list, policy_list, value_list, reward_list))


In [4]:
class Node:
    def __init__(self, state, reward, prior, muZero, args, game, parent=None, action_taken=None):
        self.state = state
        self.reward = reward
        self.children = []
        self.parent = parent
        self.total_value = 0
        self.visit_count = 0
        self.prior = prior
        self.muZero = muZero
        self.action_taken = action_taken
        self.args = args
        self.game = game

    @torch.no_grad()
    def expand(self, action_probs):
        actions = [a for a in range(self.game.action_size) if action_probs[a] > 0]
        expand_state = self.state.copy()
        expand_state = expand_state.reshape(1, 1, 3, 3).repeat(len(actions), axis=0)

        expand_state, reward = self.muZero.dynamics(
            torch.tensor(expand_state, dtype=torch.float32, device=self.muZero.device), actions)
        expand_state = expand_state.detach().cpu().numpy()
        expand_state = self.game.get_canonical_state(expand_state, -1).copy()
        
        for i, a in enumerate(actions):
            child = Node(
                expand_state[i],
                reward,
                action_probs[a],
                self.muZero,
                self.args,
                self.game,
                parent=self,
                action_taken=a,
            )
            self.children.append(child)

    def backpropagate(self, value):
        self.total_value += value
        self.visit_count += 1
        if self.parent is not None:
            self.parent.backpropagate(self.game.get_opponent_value(value))

    def is_expandable(self):
        return len(self.children) > 0

    def select_child(self):
        best_score = -np.inf
        best_child = None

        for child in self.children:
            ucb_score = self.get_ucb_score(child)
            if ucb_score > best_score:
                best_score = ucb_score
                best_child = child

        return best_child

    def get_ucb_score(self, child):
        # prior_score = child.prior * math.sqrt(self.visit_count) / (1 + child.visit_count) * (self.args['c1'] + math.log((self.visit_count + self.args['c2'] + 1) / self.args['c2']))
        prior_score = self.args['c'] * child.prior * math.sqrt(self.visit_count) / (1 + child.visit_count)
        if child.visit_count == 0:
            return prior_score
        return prior_score - (child.total_value / child.visit_count)

class MCTS:
    def __init__(self, muZero, game, args):
        self.muZero = muZero
        self.game = game
        self.args = args

    @torch.no_grad()
    def search(self, state, reward, available_actions):
        hidden_state = self.muZero.represent(
            torch.tensor(state, dtype=torch.float32, device=self.muZero.device).unsqueeze(0)
        )
        action_probs, value = self.muZero.predict(hidden_state)
        hidden_state = hidden_state.cpu().numpy().squeeze(0)
        
        root = Node(hidden_state, reward, 0, self.muZero, self.args, self.game)

        action_probs = torch.softmax(action_probs, dim=1).cpu().numpy().squeeze(0)
        action_probs = (1 - self.args['dirichlet_epsilon']) * action_probs + self.args['dirichlet_epsilon'] * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size)
        action_probs *= available_actions
        action_probs /= np.sum(action_probs)

        root.expand(action_probs)

        for simulation in range(self.args['num_mcts_runs']):
            node = root

            while node.is_expandable():
                node = node.select_child()

            action_probs, value = self.muZero.predict(
                torch.tensor(node.state, dtype=torch.float32, device=self.muZero.device).unsqueeze(0)
            )
            action_probs = torch.softmax(action_probs, dim=1).cpu().numpy().squeeze(0)
            value = value.item()

            node.expand(action_probs)
            node.backpropagate(value)

        return root

In [5]:
class MuZero(nn.Module):
    def __init__(self, game):
        super().__init__()
        self.game = game
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.predictionFunction = PredictionFunction(self.game)
        self.dynamicsFunction = DynamicsFunction()
        self.representationFunction = RepresentationFunction()

    def predict(self, hidden_state):
        return self.predictionFunction(hidden_state)

    def represent(self, observation):
        return self.representationFunction(observation)

    def dynamics(self, hidden_state, action):
        actionArr = torch.zeros((len(action), 1, 3, 3), device=self.device)
        for i, a in enumerate(action):
            actionArr[i, 0, a] = 1
        
        x = torch.cat((hidden_state, actionArr), dim=1)
        return self.dynamicsFunction(x)

# Creates hidden state + reward based on old hidden state and action 
class DynamicsFunction(nn.Module):
    def __init__(self, num_resBlocks=4, hidden_planes=64):
        super().__init__()
        
        self.startBlock = nn.Sequential(
            nn.Conv2d(2, hidden_planes, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_planes),
            nn.ReLU()
        )
        self.resBlocks = nn.ModuleList([ResBlock(hidden_planes, hidden_planes) for _ in range(num_resBlocks)])
        self.endBlock = nn.Sequential(
            nn.Conv2d(hidden_planes, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
        )
        self.rewardBlock = nn.Sequential(
            nn.Conv2d(3, 1, kernel_size=1, stride=1, padding=0),
            nn.Flatten(),
            nn.Linear(9, 1)
        )

    def forward(self, x):
        x = self.startBlock(x)
        for block in self.resBlocks:
            x = block(x)
        x = self.endBlock(x)
        reward = self.rewardBlock(x)
        return x, reward
    
# Creates policy and value based on hidden state
class PredictionFunction(nn.Module):
    def __init__(self, game, num_resBlocks=3, hidden_planes=64):
        super().__init__()
        self.game = game
        
        self.startBlock = nn.Sequential(
            nn.Conv2d(1, hidden_planes, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_planes),
            nn.ReLU()
        )
        self.resBlocks = nn.ModuleList([ResBlock(hidden_planes, hidden_planes) for _ in range(num_resBlocks)])

        self.policy_head = nn.Sequential(
            nn.Conv2d(hidden_planes, 32, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 3 * 3, self.game.action_size)
        )
        self.value_head = nn.Sequential(
            nn.Conv2d(hidden_planes, 3, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3 * 3 * 3, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.startBlock(x)
        for block in self.resBlocks:
            x = block(x)
        p = self.policy_head(x)
        v = self.value_head(x)
        return p, v

# Creates initial hidden state based on observation | several observations
class RepresentationFunction(nn.Module):
    def __init__(self, num_resBlocks=3, hidden_planes=32):
        super().__init__()

        self.l1 = nn.Linear(4, 9)
        
        self.startBlock = nn.Sequential(
            nn.Conv2d(1, hidden_planes, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_planes),
            nn.ReLU()
        )
        self.resBlocks = nn.ModuleList([ResBlock(hidden_planes, hidden_planes) for _ in range(num_resBlocks)])
        self.endBlock = nn.Sequential(
            nn.Conv2d(hidden_planes, 1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1),
        )

    def forward(self, x):
        x = self.l1(x)
        x = x.view(-1, 1, 3, 3)
        x = self.startBlock(x)
        for block in self.resBlocks:
            x = block(x)
        x = self.endBlock(x)
        return x

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, stride=stride, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual
        out = F.relu(out)
        return out


In [6]:
class Trainer:
    def __init__(self, muZero, optimizer, game, args):
        self.muZero = muZero
        self.optimizer = optimizer
        self.game = game
        self.args = args
        self.mcts = MCTS(self.muZero, self.game, self.args)
        self.replayBuffer = ReplayBuffer(self.args, self.game)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def self_play(self, game_idx):
        game_memory = []
        player = 1
        observation, valid_locations, reward, is_terminal = self.game.get_initial_state()

        while True:
            encoded_observation = self.game.get_encoded_observation(observation)
            canonical_observation = self.game.get_canonical_state(encoded_observation, player).copy()
            root = self.mcts.search(canonical_observation, reward, valid_locations)

            action_probs = [0] * self.game.action_size
            for child in root.children:
                action_probs[child.action_taken] = child.visit_count
            action_probs /= np.sum(action_probs)

            # sample action from the mcts policy | based on temperature
            if self.args['temperature'] == 0:
                action = np.argmax(action_probs)
            elif self.args['temperature'] == float('inf'):
                action = np.random.choice([r for r in range(self.game.action_size) if action_probs[r] > 0])
            else:
                temperature_action_probs = action_probs ** (1 / self.args['temperature'])
                temperature_action_probs /= np.sum(temperature_action_probs)
                action = np.random.choice(len(temperature_action_probs), p=temperature_action_probs)

            game_memory.append((canonical_observation, action, player, action_probs, reward, is_terminal))

            observation, valid_locations, reward, is_terminal = self.game.step(action)

            if is_terminal:
                return_memory = []
                for hist_state, hist_action, hist_player, hist_action_probs, hist_reward, hist_terminal in game_memory:
                    return_memory.append((
                        hist_state,
                        hist_action, 
                        hist_action_probs,
                        hist_reward,
                        game_idx,
                        hist_terminal
                    ))
                if not self.args['K'] > 0:
                    return_memory.append((
                        self.game.get_canonical_state(self.game.get_encoded_observation(observation), self.game.get_opponent_player(player)).copy(),
                        None,
                        np.zeros(self.game.action_size, dtype=np.float32),
                        0,
                        game_idx,
                        is_terminal
                    ))
                return return_memory

            player = self.game.get_opponent_player(player)

    def train(self):
        random.shuffle(self.replayBuffer.trajectories)
        for batchIdx in range(0, len(self.replayBuffer) - 1, self.args['batch_size']): 
            policy_loss = 0
            value_loss = 0
            reward_loss = 0

            observation, action, policy, value, reward = list(zip(*self.replayBuffer.trajectories[batchIdx:min(len(self.replayBuffer) -1, batchIdx + self.args['batch_size'])]))
            observation = np.stack(observation)

            state = torch.tensor(observation, dtype=torch.float32, device=self.device)
            policy = torch.tensor(np.stack(policy).swapaxes(0, 1), dtype=torch.float32, device=self.device)
            value = torch.tensor(np.array(value).swapaxes(0, 1).reshape(self.args['K'] + 1, -1, 1), dtype=torch.float32, device=self.device)

            state = self.muZero.represent(state)
            out_policy, out_value = self.muZero.predict(state)

            policy_loss += F.cross_entropy(out_policy, policy[0]) 
            value_loss += F.mse_loss(out_value, value[0])

            if self.args['K'] > 0:
                action = np.array(action).swapaxes(0, 1)

                state, out_reward = self.muZero.dynamics(state, action[0])
                observation = state.detach().cpu().numpy()
                
                reward_loss += F.mseF_loss(out_reward, reward[0])

                for k in range(1, self.args['K'] + 1):
                    observation = self.game.get_canonical_state(observation, -1).copy()
                    state = torch.tensor(observation, dtype=torch.float32, device=self.device)

                    out_policy, out_value = self.muZero.predict(state)

                    policy_loss += F.cross_entropy(out_policy, policy[k])
                    value_loss += F.mse_loss(out_value, value[k])

                    state, out_reward = self.muZero.dynamics(state, action[k])
                    observation = state.detach().cpu().numpy()

                    reward_loss += F.mse_loss(out_reward, reward[k])

            loss = value_loss * self.args['value_loss_weight'] + policy_loss + reward_loss
            loss /= self.args['K'] + 1

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

    def run(self):
        for iteration in range(self.args['num_iterations']):
            print(f"iteration: {iteration}")
            self.replayBuffer.empty()

            self.muZero.eval()
            for train_game_idx in trange(self.args['num_train_games'], desc="train_game"):
                self.replayBuffer.memory += self.self_play(train_game_idx + iteration * self.args['num_train_games'])
            self.replayBuffer.build_trajectories()

            self.muZero.train()
            for epoch in trange(self.args['num_epochs'], desc="epochs"):
                self.train()

            torch.save(self.muZero.state_dict(), f"Models/{self.game}/model_{iteration}.pt")
            torch.save(self.optimizer.state_dict(), f"Models/{self.game}/optimizer_{iteration}.pt")


In [8]:
args = {
    'num_iterations': 10,
    'num_train_games': 100,
    'num_mcts_runs': 800,
    'num_epochs': 4,
    'batch_size': 64,
    'temperature': 1,
    'K': 5,
    'c': 2,
    'N': 10,
    'dirichlet_alpha': 0.3,
    'dirichlet_epsilon': 0.25,
    'discount': 0.997,
    'value_loss_weight': 0.25,
}

game = CartPole()
muZero = MuZero(game).to(device)
optimizer = torch.optim.Adam(muZero.parameters(), lr=0.001)
trainer = Trainer(muZero, optimizer, game, args)
trainer.run()

iteration: 0


train_game:   0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [64, 1, 3, 3], expected input[1, 3, 3, 3] to have 1 channels, but got 3 channels instead

In [ ]:
import gymnasium as gym

env = gym.make('LunarLander-v2', render_mode="human")

observation, info = env.reset(seed=42)
print(observation)
for _ in range(100):
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()